### Set up environnent, access to google drive, import librairies


In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [3]:
pip install keras --upgrade

     |████████████████████████████████| 378kB 9.8MB/s 
  Found existing installation: Keras 2.2.5
    Uninstalling Keras-2.2.5:
      Successfully uninstalled Keras-2.2.5


In [4]:
import os, sys
sys.path.insert(0, os.path.abspath('/content/drive/My Drive/ML_Project_2/'))
from models import *
from helpers import *

path_g = '/content/drive/My Drive/ML_Project_2/'

Using TensorFlow backend.


In [0]:
import numpy as np
import time
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
import gensim
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, Callback
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split, KFold, cross_val_score
import tensorflow as tf
import keras
import pandas as pd

In [6]:
# Load full dataset or not: 'f' or 'nf'
full='f'
processed=False

if processed:
    text_data = np.load('/content/drive/My Drive/ML_Project_2/Processed_Data/data_train_pr_' + full + '_sl5' + '.npy', allow_pickle=True)
    text_data_test = np.load('/content/drive/My Drive/ML_Project_2/Processed_Data/data_test_pr_sl5' + '.npy', allow_pickle=True)
    labels = np.load('/content/drive/My Drive/ML_Project_2/Processed_Data/labels_train_'+ full +'_sl5.npy')
    dataset_type = 'processed'

else:
    text_data, labels, text_data_test = get_raw_data(path_g, full)
    dataset_type = 'raw'


(1142838, 2)
(1127644, 2)


In [0]:
perm = np.random.permutation(text_data.shape[0])
text_data = text_data[perm]
labels = labels[perm]

# If we don't want to take full file
n_train = -1

if n_train > 0:
    text_data = text_data[:n_train]
    labels = labels[:n_train]


### Train or load gensim Word2Vec model

In [0]:
train_w2v = True
if train_w2v:
  # Take all dataset to train gensim word2vec
  text_data_tot = np.concatenate((text_data, text_data_test), axis=0)

  t1 = time.time()

  # Define gensim model
  size_w2v = 400
  iter_w2v = 5
  window_w2v = 5
  min_count = 6


  # Name to save the model afterwards
  path_w2v = '/content/drive/My Drive/ML_Project_2/w2v_models/'
  name_w2v = 'w2v_f_s' + str(size_w2v) + '_i' + str(iter_w2v) + '_w' + \
              str(window_w2v) + '_mc' + str(min_count) + '_full_' + full + '_' + dataset_type

  model_gs = gensim.models.Word2Vec(text_data_tot, size=size_w2v, window=window_w2v, sg=1,\
                                    min_count=min_count, iter=iter_w2v)
  word_vector = model_gs.wv
  print("Total time to train gensim", time.time() - t1, "s", flush=True)
  word_vector.save(path_w2v+name_w2v)


else:
  
  path_w2v = '/content/drive/My Drive/ML_Project_2/w2v_models/'
  name_w2v = 'w2v_f_s' + str(size_w2v) + '_i' + str(iter_w2v) + '_w' + \
              str(window_w2v) + '_mc' + str(min_count) + '_full_' + full + '_' + dataset_type'
  word_vector = gensim.models.KeyedVectors.load(path_w2v + name_w2v)


In [0]:
# Convert gensim word_vector in keras embedding
train_emb = False
k_emb = word_vector.get_keras_embedding(train_embeddings=train_emb)
size_emb = k_emb.output_dim

In [0]:
# Convert text to numerical data according to gensim (now keras embedding) vocabulary 
vocabulary = {word: vector.index for word, vector in word_vector.vocab.items()}
tk = Tokenizer(num_words=len(vocabulary))
tk.word_index = vocabulary
num_data = np.asarray((pad_sequences(tk.texts_to_sequences(text_data), padding='post')))
num_data_test = np.asarray((pad_sequences(tk.texts_to_sequences(text_data_test), 
                                          maxlen=num_data.shape[1],padding='post')))

##Hyperparameters Tuning procedure:
First, the size of the embedding and the number of filters were conjointly tuned (using fixed but reasonable values for other hyper-parameters).

Then, using the best combination for size_emb and n_filters was taken, and hidden-dims were optimized.

And finally the batch-size (what can be seen here)

In [0]:
class GetAcc(Callback):
    def __init__(self, data_test, labels_test, acc_array, kfold_idx):
        self.data_test = data_test
        self.labels_test = labels_test
        self.acc = acc_array
        self.kfold_idx = kfold_idx

    def on_epoch_end(self, epoch, logs={}):
        loss, accuracy = self.model.evaluate(self.data_test, self.labels_test, verbose=0)
        self.acc[self.kfold_idx, epoch] = accuracy
        #print('kfold_idx:', self.kfold_idx, ' Epochs #', epoch)
        #print('\nTesting loss: {}, acc: {}\n'.format(loss, accuracy))

In [0]:
%%script false 
filters, kernel_size, batch_size = 300, 5, 150
epochs = 8
hidden_dims =  250
learning_rate =  0.001
dropout = 0.2

model = build_model_emb(k_emb, filters, kernel_size, hidden_dims, num_data.shape[1], 
                        size_emb, learning_rate, dropout=dropout)

results = pd.DataFrame(columns=['ed', 'train_emb', 'nf', 'ks', 'hd', 'dp', 'lr', 'bs', 'ep', 'accuracy', 'mean_accuracy', 'std_accuracy'])
# For loop
n_splits = 5

optim_param = 'batch_size'
batch_size = [32, 64, 150, 250]

for batch_size in batch_size:
    acc_array = np.zeros((n_splits, epochs))
    k_i = 0
    print('batch_size:', batch_size)
    for train_index, test_index in KFold(n_splits).split(num_data):
        print('k_fold:', k_i)
        x_train, x_test = num_data[train_index], num_data[test_index]
        y_train, y_test = labels[train_index], labels[test_index]    
        
        k_emb = word_vector.get_keras_embedding(train_embeddings=train_emb)

        model = build_model_emb(k_emb, filters, kernel_size, hidden_dims, num_data.shape[1], 
                        size_emb, learning_rate, dropout=dropout)
        
        model.fit(x_train, y_train,
                batch_size=batch_size,
                epochs=epochs,
                validation_data=(x_test, y_test),
                callbacks=[GetAcc(x_test, y_test, acc_array, k_i)])
        k_i += 1

    for n in range(epochs):
        results = results.append({'ed': size_emb, 'train_emb': train_emb,'nf': filters, 'ks': kernel_size, 'hd': hidden_dims, 
                                    'dp': dropout, 'lr':learning_rate, 'bs': batch_size, 'ep': n+1,
                                    'accuracy': acc_array[:, n], 'mean_accuracy': np.mean(acc_array[:, n]), 
                                    'std_accuracy': np.std(acc_array[:, n])}, ignore_index=True)
        


In [0]:
%%script false 
path = '/content/drive/My Drive/ML_Project_2/'
name = 'Results_DF/res_'+ 'emb_' + str(size_emb) + '_tr_'+ str(train_emb) + '_opt_' + optim_param + '.txt'
print(name)
if not os.path.exists(path + name):
    results.to_csv(path + name)

else:
    print("Path already existing !! Change name for result")

In [0]:
%%script false 
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(results.loc[:, results.columns != 'accuracy'])

In [0]:
%%script false 
imax = np.argmax(results['mean_accuracy'].values)
print("Best: %f using \n %s" % (results['mean_accuracy'].iloc[imax], results.iloc[imax]))

### Cross validation with bigger number of k-fold with the different datasets: processed and raw

In [0]:
filters, kernel_size, batch_size = 300, 5, 150
epochs = 8
hidden_dims =  250
learning_rate =  0.001
dropout = 0.2

n_splits = 10

results_2 = pd.DataFrame(columns=['dataset', 'ed', 'train_emb', 'nf', 'ks', 'hd', 'dp', 'lr', 'bs', 'ep', 'accuracy', 'mean_accuracy', 'std_accuracy'])
acc_array = np.zeros((n_splits, epochs))



In [0]:
k_i = 0


for train_index, test_index in KFold(n_splits).split(num_data):
    print('k_fold:', k_i)
    x_train, x_test = num_data[train_index], num_data[test_index]
    y_train, y_test = labels[train_index], labels[test_index]
    k_emb = word_vector.get_keras_embedding(train_embeddings=train_emb)
    model = build_model_emb(k_emb, filters, kernel_size, hidden_dims, num_data.shape[1], 
                        size_emb, learning_rate, dropout=dropout)
    model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_data=(x_test, y_test),
                    callbacks=[GetAcc(x_test, y_test, acc_array, k_i)])
    k_i += 1

for n in range(epochs):
    results_2 = results_2.append({'dataset': dataset_type,'ed': size_emb, 'train_emb': train_emb,'nf': filters, 'ks': kernel_size, 'hd': hidden_dims, 
                                'dp': dropout, 'lr':learning_rate, 'bs': batch_size, 'ep': n+1,
                                'accuracy': acc_array[:, n], 'mean_accuracy': np.mean(acc_array[:, n]), 
                                'std_accuracy': np.std(acc_array[:, n])}, ignore_index=True)


In [0]:
print(results_2)

In [0]:
path = '/content/drive/My Drive/ML_Project_2/'
name = 'Results_DF/res_'+ 'emb_' + str(size_emb) + '_tr_'+ str(train_emb) + '_' +dataset_type + '_final' + '.txt'
print(name)
if not os.path.exists(path + name):
    results_2.to_csv(path + name)

else:
    print("Path already existing !! Change name for result")